**Import packages**

In [1]:
import warnings
# Import TF and TF Hub libraries.
import tensorflow as tf
import tensorflow_hub as hub
#GUI
import PySimpleGUI as sg
import os, csv, tqdm, cv2, time
from PIL import Image
import numpy as np
import pandas as pd
import shutil
# Import Classifiers
from sklearn.model_selection import train_test_split as tts
from sklearn import svm
from sklearn import metrics

import mediapipe as mp
from mediapipe.python.solutions import hands as handsModule

In [2]:
warnings.filterwarnings("ignore")

**Path to directories**

In [3]:
directory = 'C:\\Users\\rache\\Documents\\'

In [4]:
# Path to the folder containing the input images
images_folder = directory +'Rehabilitation_interface\\HandsEvaluation\\Data\\' #1
# Path to the folder containing the body keypoints
images_out_folder = directory +'Rehabilitation_interface\\HandsEvaluation\\DataOutputMovenet\\' #2
# Path to the resized images
resized_path = directory +'Rehabilitation_interface\\HandsEvaluation\\resized\\' #3
# Path to the output CSV file containing the body keypoints
csv_out_path = directory +'Rehabilitation_interface\\HandsEvaluation\\DataOutputMovenet\\fitness_poses_csvs_out_basic_movenet.csv' #4
# Path to the images
path_to_data = directory +'Rehabilitation_interface\\HandsEvaluation\\Data'
path_to_classes = directory +'Rehabilitation_interface\\HandsEvaluation\\Data\\clase'

**Initializing the Model**

In [5]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)

**Functions of the interface**

First GUI

In [6]:
#Layout definition
def create_classes_layout():
    ## Define the layout for the "Create Classes" window
    layout = [
        [sg.Text('Number of classes:'), sg.InputText(default_text='2',size=(10,1))],
        [sg.Text('Number of photos per class:'), sg.InputText(default_text='5',size=(10,1))],
        [sg.Button('Create Classes')],
        [sg.Text(size=(40,1), key='-OUTPUT-')],
        [sg.Image(filename='', key='-IMAGE-', size=(30,15))],
        [sg.Button('Close window')],
    ]
    return layout

#Creation of windows
def create_classes_window():
    ## Create the "Create Classes" window
    layout = create_classes_layout()
    window = sg.Window('Pose Classifier - Create Classes', layout)
    return window

#Creation of the classes with the interface
def create_classes(window, path_to_data, path_to_classes, num_classes, num_photos, time_between_photos):
    # Remove all directories that start with "clase"
    for root, dirs, files in os.walk(path_to_data):
        for dir in dirs:
            if dir.startswith("clase"):
                shutil.rmtree(os.path.join(root, dir))
    for clase in range(num_classes):
        # Wait for the user to be ready
        sg.popup('Haz la posición número {}. Pulsa OK cuando estés listo.'.format(clase+1), title='¡Listo!')
        window['-OUTPUT-'].update('Haz la posición número {}. Esperando...'.format(clase+1))
        window.refresh()
        time.sleep(3)
        folder_name = str(path_to_classes+str(clase))
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        # Loop to take the photos
        for i in range(num_photos):
            window['-OUTPUT-'].update('Foto {} de {}'.format(i+1,num_photos))
            window.refresh()
            # Read a frame from the webcam
            ret, frame = camera.read()
            # Convert the image to PNG Bytes
            resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            imgbytes = cv2.imencode('.png', resized)[1].tobytes()
            # Check if the camera is working
            if not ret:
                window['-OUTPUT-'].update('La cámara no funciona!')
                window.refresh()
                break
            # Update the image element in the GUI with the new image
            window['-IMAGE-'].update(data=imgbytes)
            # Generate the file name for the current picture
            filename = folder_name+'/photo_{}.png'.format(i)
            # Save the frame to a file
            cv2.imwrite(filename, frame)
            # Wait for the specified time before taking the next photo
            time.sleep(time_between_photos)

Second GUI

In [7]:
# Change size images to see them on the interface 
def class_image_resized():
    image_paths = []

    for folder_name in os.listdir(images_folder):
        if folder_name.startswith("clase"):
            image_paths.append(os.path.join(images_folder, folder_name, "photo_0.png"))
    for i, image_path in enumerate(image_paths):
        img = Image.open(image_path)
        img_resized = img.resize((100, 100))
        resized_image_path = os.path.join(resized_path, f"clase{i}.png")
        img_resized.save(resized_image_path)
    
    image_columns = []
    for i in range(len(image_paths)):
        image_column = sg.Column([[sg.Image(filename = os.path.join(resized_path,f"clase{i}.png"))]])#, pad = (0,0))
        image_columns.append(image_column)
    return image_columns

#Buttons to train and classify the models
def create_training_layout():
    image_columns = class_image_resized()
    # Creamos un layout con una fila y las columnas de las imágenes
    layout = [image_columns]
    # Add remaining GUI elements
    layout += [
        [sg.Text('The rehabiliation is fixed on Full body')],
        #[sg.Checkbox('Full body:', default=True, key="-Full body-"),sg.Checkbox('Cara:', default=False, key="-Cara-"),sg.Checkbox('Torso:', default=False, key="-Torso-"),sg.Checkbox('Piernas:', default=False, key="-Piernas-"),sg.Checkbox('Puntos fiables:', default=False, key="-Puntos fiables-")],
        [sg.Button('Train Model')],
        [sg.Text(size=(50,1), key='-1-')],
        [sg.Button('Classify Pose from Camera')],
        [sg.Text(size=(50,1), key='-2-')],
        #[sg.Button('Try Real Time'), sg.Button('STOP', button_color=('white', 'red'))],
        #[sg.Text(size=(50,1), key='-3-')],
        [sg.Button('Close Window')]
    ]
    return layout

#Corret format of the indexes
def keypoints_format(list_points):
    #list_points = [list_points[i]-1 for i in range(len(list_points))]
    idx = pd.Index(list(map(str, list_points)))
    return idx
def keypoints_in_range(list_points):
    new_idx = [int(list_points[i])-1 for i in range(len(list_points))]
    updated_idx = pd.Index(new_idx)
    return updated_idx

#Choose keypoints to detect
def keypoints_per_part_of_body(values):
    df = pd.read_csv(csv_out_path)    
    list_points = [1, 2, 3, 4, 5, 6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
    idx = keypoints_format(list_points)    
    numbers = df.columns.str.extract(r'(x|y)(\d+)')[1] #obtenemos los valores numericos de las columnas
    cols_to_keep = df[df.columns[df.columns.isin(['name', 'class']) | numbers.isin(idx)]]
    cols_to_keep.to_csv(csv_out_path, index=False) #guardamos el dataframe final en el fichero csv_out_path sobrescribiéndolo
    df = pd.read_csv(csv_out_path)
    display(df)
    idx = keypoints_in_range(idx)
    return idx
    
#Second windows
def create_training_window():
    ## Create the "Create Classes" window
    layout = create_training_layout()
    window = sg.Window('Ejemplo de imágenes', layout)
    return window

# Define the function to extract landmarks from an image
def extract_landmarks(image_path):
    # Load image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb) 
    # If hands are present in the image (frame)
    if results.multi_hand_landmarks:
        # Both hands are present in the image (frame)
        if len(results.multi_handedness) == 2:
            keypoints = results.multi_hand_landmarks
            return keypoints
        
# Extract features from images
def extract_features_to_train(csv_out_file, images_folder, images_out_foulder):
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_folder) if not n.startswith('.')])
    # Write the header row to the CSV file
    csv_out_writer.writerow(['name', 'class'] + [f'x{i}' for i in range(1, 43)] + [f'y{i}' for i in range(1, 43)] + [f'z{i}' for i in range(1, 43)])
    for pose_class_name in pose_class_names:
        message = f'Bootstrapping {pose_class_name}'
        window['-1-'].update(message)
        if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
            os.makedirs(os.path.join(images_out_folder, pose_class_name))
        image_names = sorted([
        n for n in os.listdir(os.path.join(images_folder, pose_class_name))
        if not n.startswith('.')])
        new_row = []
        for image_name in tqdm.tqdm(image_names, position=0):
            new_row = []
            # Write pose sample to CSV.
            new_row.append(image_name)
            new_row.append(pose_class_name)
            # Extract the landmarks from the image
            landmarks = extract_landmarks(os.path.join(images_folder,pose_class_name, image_name))
            # Assuming you have the pose data in a variable named `pose`
            #print("landmarks", landmarks)
            if landmarks is not None:
                # 1. Convert landmarks data to a list of dictionaries
                landmarks_data = []
                #for landmark in landmarks:
                for handLandmarks in landmarks:
                    for point in handsModule.HandLandmark:
                        normalizedLandmark = handLandmarks.landmark[point]
                        # process each landmark
                        # process each hand list of landmark
                        #print("landmark", normalizedLandmark)
                        new_row.append(normalizedLandmark.x)
                        new_row.append(normalizedLandmark.y)
                        new_row.append(normalizedLandmark.z)
            # Save the landmarks to the CSV file
            #print(new_row)
            csv_out_writer.writerow(new_row)
    # Aqui quiero cambiar todo el fichero csv_out_path y sobreescribirlo
    csv_out_file.close()
    #return idx
    idx = keypoints_per_part_of_body(values)
    return idx

#Classify the images
def classify_from_data():
    df = pd.read_csv(csv_out_path)
    y = df["class"]
    x = df.loc[:,df.columns!="class"]
    x = x.loc[:,x.columns!="name"]
    ###Splitting train/test data
    display(x)
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=25/100,random_state=109)
    ###Creating Support Vector Machine Model
    clf = svm.SVC(kernel='rbf')
    ###Training the Model
    clf.fit(X_tr, y_tr)
    ###Making Predictions
    y_pr = clf.predict(X_tst)
    return clf, y_tst, y_pr

In [8]:
## First window
window = create_classes_window()
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if event == 'Create Classes':
        window['-OUTPUT-'].update('Creating classes...')
        window.refresh()
        # Initialize the webcam
        camera = cv2.VideoCapture(0)
        # Close if we try to crate new data without specifying the size
        try:
            num_classes = int(values[0])
            num_photos = int(values[1])
        except ValueError:
            window.close()
            break
        # Define the number of photos to take and the time between photos
        num_classes = int(values[0])
        num_photos = int(values[1])
        time_between_photos = 0.5
        create_classes(window, path_to_data, path_to_classes, num_classes, num_photos, time_between_photos)       
        # Release the webcam
        camera.release()
        window['-OUTPUT-'].update('Classes created')
        time.sleep(2)
        window.close()
    elif event == 'Close window':
        break        
# Close the window when the event loop is exited
window.close()

#Second window
window = create_training_window()
stop_flag = True
while True:
    event, values = window.read()
    if event == sg.WINDOW_CLOSED:
        break
    elif event == 'Train Model':
        with open(csv_out_path, 'w') as csv_out_file:
            idx = extract_features_to_train(csv_out_file, images_folder, images_out_folder)
            #print('idx', idx)
            window['-1-'].update('Model trained')
    elif event == 'Classify Pose from Camera':
        ###Evaluating Prediction Accuracy
        clf, y_tst, y_pr = classify_from_data()
        window['-2-'].update(f'Pose classified from camera. Accuracy: {metrics.accuracy_score(y_tst, y_pr)}')
        print('Pose classified from camera. Accuracy:', metrics.accuracy_score(y_tst, y_pr))
    elif event == 'Close window':
        break  
window.Close()

**Create test images**

#Layout definition
def create_classes_layout():
    ## Define the layout for the "Create Classes" window
    layout = [
        [sg.Text('Number of classes:'), sg.InputText(default_text='2',size=(10,1))],
        [sg.Text('Number of photos per class:'), sg.InputText(default_text='5',size=(10,1))],
        [sg.Button('Create Classes')],
        [sg.Text(size=(40,1), key='-OUTPUT-')],
        [sg.Image(filename='', key='-IMAGE-', size=(30,15))],
        [sg.Button('Close window')],
    ]
    return layout

#Creation of windows
def create_classes_window():
    ## Create the "Create Classes" window
    layout = create_classes_layout()
    window = sg.Window('Pose Classifier - Create Classes', layout)
    return window

#Creation of the classes with the interface
def create_classes(window, path_to_data, path_to_classes, num_classes, num_photos, time_between_photos):
    # Remove all directories that start with "clase"
    for root, dirs, files in os.walk(path_to_data):
        for dir in dirs:
            if dir.startswith("clase"):
                shutil.rmtree(os.path.join(root, dir))
    for clase in range(num_classes):
        # Wait for the user to be ready
        sg.popup('Haz la posición número {}. Pulsa OK cuando estés listo.'.format(clase+1), title='¡Listo!')
        window['-OUTPUT-'].update('Haz la posición número {}. Esperando...'.format(clase+1))
        window.refresh()
        time.sleep(3)
        folder_name = str(path_to_classes)#+str(clase))
        #if not os.path.exists(folder_name):
        #    os.makedirs(folder_name)
        # Loop to take the photos
        for i in range(num_photos):
            window['-OUTPUT-'].update('Foto {} de {}'.format(i+1,num_photos))
            window.refresh()
            # Read a frame from the webcam
            ret, frame = camera.read()
            # Convert the image to PNG Bytes
            resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
            imgbytes = cv2.imencode('.png', resized)[1].tobytes()
            # Check if the camera is working
            if not ret:
                window['-OUTPUT-'].update('La cámara no funciona!')
                window.refresh()
                break
            # Update the image element in the GUI with the new image
            window['-IMAGE-'].update(data=imgbytes)
            # Generate the file name for the current picture
            filename = folder_name+'/photo_{}.png'.format(i)
            # Save the frame to a file
            cv2.imwrite(filename, frame)
            # Wait for the specified time before taking the next photo
            time.sleep(time_between_photos)

Test images layout creation

In [9]:
# Path to the folder containing the input images
#images_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\Data\\' #1
# Path to the folder containing the body keypoints
#images_out_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\DataOutputMovenet\\' #2
# Path to the output CSV file containing the body keypoints
csv_out_path_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\fitness_poses_csvs_out_basic_movenet.csv' #4
# Path to the images
path_to_data_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\Data'
path_to_classes_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\Data'
# Path to the folder containing the input images
images_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\' #1
images_out_folder_test = directory +'Rehabilitation_interface\\HandsEvaluation\\test\\' #2
# Event loop to process GUI events
window = create_classes_window()
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    if event == 'Create Classes':
        window['-OUTPUT-'].update('Creating classes...')
        window.refresh()
        # Initialize the webcam
        camera = cv2.VideoCapture(0)
        # Close if we try to crate new data without specifying the size
        try:
            num_classes = int(values[0])
            num_photos = int(values[1])
        except ValueError:
            window.close()
            break
        # Define the number of photos to take and the time between photos
        num_classes = int(values[0])
        num_photos = int(values[1])
        time_between_photos = 0.5
        create_classes(window, path_to_data_test, path_to_classes_test, num_classes, num_photos, time_between_photos)       
        # Release the webcam
        camera.release()
        window['-OUTPUT-'].update('Classes created')
        time.sleep(2)
        window.close()
    elif event == 'Close window':
        break        

In [10]:
#TEST GENERATING CSV
def extract_features_to_test(csv_out_file, images_folder, images_out_foulder):
    csv_out_writer = csv.writer(csv_out_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    # Folder names are used as pose class names.
    pose_class_names = sorted([n for n in os.listdir(images_folder) if not n.startswith('.')])
    # Write the header row to the CSV file
    csv_out_writer.writerow(['name', 'class'] + [f'x{i}' for i in range(1, 43)] + [f'y{i}' for i in range(1, 43)] )#+ [f'z{i}' for i in range(1, 43)])
    image_names = sorted([
    n for n in os.listdir(os.path.join(images_folder))#, pose_class_name))
    if not n.startswith('.')])
    new_row = []
    pose_class_name = ''
    #print(image_names)
    for image_name in tqdm.tqdm(image_names[1:], position=0):      
        new_row = []
        # Write pose sample to CSV.
        new_row.append(image_name)
        new_row.append('clase'+image_name[6:7])
        # Extract the landmarks from the image
        landmarks = extract_landmarks(os.path.join(images_folder,pose_class_name, image_name))
        # Assuming you have the pose data in a variable named `pose`
        if landmarks is not None:
            # 1. Convert landmarks data to a list of dictionaries
            landmarks_data = []
            for handLandmarks in landmarks:
                for point in handsModule.HandLandmark:
                    normalizedLandmark = handLandmarks.landmark[point]
                    # process each landmark
                    new_row.append(normalizedLandmark.x)
                    new_row.append(normalizedLandmark.y)
        # Save the landmarks to the CSV file
        csv_out_writer.writerow(new_row)
    # Aqui quiero cambiar todo el fichero csv_out_path y sobreescribirlo
    csv_out_file.close()

with open(csv_out_path_test, 'w') as csv_out_file_test:
    idx_T = extract_features_to_test(csv_out_file_test, images_folder_test, images_out_folder_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.60it/s]


In [11]:
pd.read_csv(csv_out_path_test).head()

,name,class,x1,x2,x3,x4,x5,x6,x7,x8,...,y33,y34,y35,y36,y37,y38,y39,y40,y41,y42
0,photo_0.png,clase0,0.762691,0.834830,0.689139,0.786303,0.650712,0.699587,0.630378,0.632270,...,0.328748,0.630156,0.287858,0.581328,0.319657,0.558810,0.313797,0.611363,0.298750,0.618244
1,photo_1.png,clase1,0.764203,0.837344,0.687731,0.786719,0.650168,0.696513,0.632724,0.628378,...,0.328521,0.631966,0.288051,0.582325,0.320274,0.559385,0.315830,0.608413,0.301320,0.618250
2,photo_2.png,clase2,0.740577,0.871423,0.704638,0.834968,0.689132,0.746378,0.725358,0.688177,...,0.320575,0.690687,0.262407,0.623886,0.288635,0.688484,0.296029,0.718996,0.290364,0.704069
3,photo_3.png,clase3,0.732781,0.865792,0.689244,0.828949,0.667072,0.749219,0.695606,0.696842,...,0.306524,0.707875,0.253100,0.629664,0.272555,0.690159,0.284051,0.725848,0.284318,0.718668
4,photo_4.png,clase4,0.728929,0.884306,0.674763,0.839428,0.647238,0.755053,0.676080,0.688686,...,0.301085,0.743056,0.244236,0.657492,0.263733,0.744643,0.274443,0.766850,0.276269,0.746181


## Different algorithms accuracy

### SVM

In [13]:
import pandas as pd
from sklearn import metrics
from itertools import combinations
#SVM
def classify_from_data_df_svm(df):
    y = df["class"]
    x = df.loc[:,df.columns!="class"]
    x = x.loc[:,x.columns!="name"]
    ###Splitting train/test data
    #display(x)
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=25/100,random_state=109)
    ###Creating Support Vector Machine Model
    clf = svm.SVC(kernel='rbf')
    ###Training the Model
    clf.fit(X_tr, y_tr)
    ###Making Predictions
    y_pr = clf.predict(X_tst)
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_svm(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        acc.append(accuracy)
        accTemp.append(accuracy)
    print("HAND SVM: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("HAND SVM: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv(csv_out_path_test)
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = svm.SVC(kernel='rbf')

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

HAND SVM: Mean accuracy with all the combinations of  2  classes:  0.7733333333333331
HAND SVM: Mean accuracy with all the combinations of  3  classes:  0.3333333333333333
HAND SVM: Mean accuracy with all the combinations of  4  classes:  0.3799999999999999
HAND SVM: Mean accuracy with all the combinations of  5  classes:  0.23595848595848593
HAND SVM: Mean accuracy with all the combinations of  6  classes:  0.15619047619047619
HAND SVM: Mean accuracy with all the combinations of  7  classes:  0.39351851851851866
HAND SVM: Mean accuracy with all the combinations of  8  classes:  0.18666666666666665
HAND SVM: Mean accuracy with all the combinations of  9  classes:  0.19565217391304346
HAND SVM: Mean accuracy with all the combinations of  10  classes:  0.36
--------------------
HAND SVM: Mean accuracy of ALL the combinations of classes:  0.30088872894658547
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
8    cla

### KNN

In [14]:
from sklearn.neighbors import KNeighborsClassifier

def classify_from_data_df_knn(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating KNN model
    clf = KNeighborsClassifier(n_neighbors=5)
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_knn(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        acc.append(accuracy)
        accTemp.append(accuracy)
    print("HAND KNN: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("HAND KNN: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv(csv_out_path_test)
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = KNeighborsClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

HAND KNN: Mean accuracy with all the combinations of  2  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  3  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  4  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  5  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  6  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  7  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  8  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  9  classes:  1.0
HAND KNN: Mean accuracy with all the combinations of  10  classes:  1.0
--------------------
HAND KNN: Mean accuracy of ALL the combinations of classes:  1.0
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
8    clase8
9    clase9
Name: class, dtype: object
y_test_pred ['clase0' 'clase0' 'clase0' 'clase0' 'clase0' 'clase0' 'clase0' 'clase0'
 'clase0' 'cla

### DECISION TREE

In [15]:
from sklearn.tree import DecisionTreeClassifier

def classify_from_data_df_decision_tree(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating Decision Tree model
    clf = DecisionTreeClassifier()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr

# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_decision_tree(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        acc.append(accuracy)
        accTemp.append(accuracy)
    print("HAND DT: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("HAND DT: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv(csv_out_path_test)
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = DecisionTreeClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

HAND DT: Mean accuracy with all the combinations of  2  classes:  0.9911111111111112
HAND DT: Mean accuracy with all the combinations of  3  classes:  0.9447916666666667
HAND DT: Mean accuracy with all the combinations of  4  classes:  0.9809523809523809
HAND DT: Mean accuracy with all the combinations of  5  classes:  0.9627594627594628
HAND DT: Mean accuracy with all the combinations of  6  classes:  0.9425396825396826
HAND DT: Mean accuracy with all the combinations of  7  classes:  0.927314814814815
HAND DT: Mean accuracy with all the combinations of  8  classes:  0.9055555555555557
HAND DT: Mean accuracy with all the combinations of  9  classes:  0.9478260869565218
HAND DT: Mean accuracy with all the combinations of  10  classes:  0.92
--------------------
HAND DT: Mean accuracy of ALL the combinations of classes:  0.9545407271431994
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
8    clase8
9    clase9
N

### ADA BOOST CLASSIFIER

In [16]:
from sklearn.ensemble import AdaBoostClassifier

def classify_from_data_df_adaboost(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating AdaBoost model
    clf = AdaBoostClassifier()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr
# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_adaboost(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        acc.append(accuracy)
        accTemp.append(accuracy)
    print("HAND AB: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("HAND AB: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv(csv_out_path_test)
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = AdaBoostClassifier()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

HAND AB: Mean accuracy with all the combinations of  2  classes:  0.9955555555555555
HAND AB: Mean accuracy with all the combinations of  3  classes:  0.9989583333333333
HAND AB: Mean accuracy with all the combinations of  4  classes:  0.9995238095238096
HAND AB: Mean accuracy with all the combinations of  5  classes:  0.9929792429792431
HAND AB: Mean accuracy with all the combinations of  6  classes:  0.9736507936507938
HAND AB: Mean accuracy with all the combinations of  7  classes:  0.9361111111111112
HAND AB: Mean accuracy with all the combinations of  8  classes:  0.9411111111111111
HAND AB: Mean accuracy with all the combinations of  9  classes:  0.8956521739130435
HAND AB: Mean accuracy with all the combinations of  10  classes:  0.8
--------------------
HAND AB: Mean accuracy of ALL the combinations of classes:  0.9809598133957548
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
8    clase8
9    clase9
N

### NAIVE BAYES

In [17]:
from sklearn.naive_bayes import GaussianNB

def classify_from_data_df_naive_bayes(df):
    y = df["class"]
    x = df.loc[:, df.columns != "class"]
    x = x.loc[:, x.columns != "name"]
    
    # Splitting train/test data
    X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)
    
    # Creating Naive Bayes model
    clf = GaussianNB()
    
    # Training the model
    clf.fit(X_tr, y_tr)
    
    # Making predictions
    y_pr = clf.predict(X_tst)
    
    return clf, y_tst, y_pr
# Leer el dataframe
df = pd.read_csv(csv_out_path)

# Obtener todas las clases únicas
classes = df['class'].unique()

acc = []
accTemp = []
# Realizar la evaluación para todas las combinaciones posibles
for r in range(2, len(classes) + 1):
    combinations_r = combinations(classes, r)
    for combination in combinations_r:
        # Filtrar el dataframe para las clases seleccionadas
        df2 = df[df['class'].isin(combination)]
        
        # Realizar la clasificación y calcular la precisión
        clf, y_tst, y_pr = classify_from_data_df_naive_bayes(df2)
        accuracy = metrics.accuracy_score(y_tst, y_pr)
        
        # Imprimir los resultados
        class_str = ', '.join(combination)
        #print(f'Pose classified from camera. Clases: {class_str}. Accuracy: {accuracy}')
        acc.append(accuracy)
        accTemp.append(accuracy)
    print("HAND NB: Mean accuracy with all the combinations of ", r, " classes: ", np.mean(accTemp))
    accTemp = []
        
print("--------------------")
print("HAND NB: Mean accuracy of ALL the combinations of classes: ", np.mean(acc))
df_test = pd.read_csv(csv_out_path_test)
y_test = df_test["class"]
x_test = df_test.loc[:, df_test.columns != "class"]
x_test = x_test.loc[:, x_test.columns != "name"]
# Making test predictions
y = df["class"]
x = df.loc[:, df.columns != "class"]
x = x.loc[:, x.columns != "name"]

# Splitting train/test data
X_tr, X_tst, y_tr, y_tst = tts(x, y, test_size=0.25, random_state=109)

# Creating AdaBoost model
clf = GaussianNB()

# Training the model
clf.fit(X_tr, y_tr)
y_test_pred = clf.predict(x_test)
accuracy = metrics.accuracy_score(y_test, y_test_pred)
print("--------------------")
print('y_test', y_test)
print('y_test_pred', y_test_pred)
print("--------------------")
print('Accuracy on new test data: ', accuracy)

HAND NB: Mean accuracy with all the combinations of  2  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  3  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  4  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  5  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  6  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  7  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  8  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  9  classes:  1.0
HAND NB: Mean accuracy with all the combinations of  10  classes:  1.0
--------------------
HAND NB: Mean accuracy of ALL the combinations of classes:  1.0
--------------------
y_test 0    clase0
1    clase1
2    clase2
3    clase3
4    clase4
5    clase5
6    clase6
7    clase7
8    clase8
9    clase9
Name: class, dtype: object
y_test_pred ['clase6' 'clase6' 'clase6' 'clase8' 'clase8' 'clase8' 'clase6' 'clase6'
 'clase6' 'clase6']
----